## Machine Learning- Exoplanet Exploration

#### Extensive Data Dictionary: https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html

Highlightable columns of note are:

* kepoi_name: A KOI is a target identified by the Kepler Project that displays at least one transit-like sequence within Kepler time-series photometry that appears to be of astrophysical origin and initially consistent with a planetary transit hypothesis

* kepler_name: [These names] are intended to clearly indicate a class of objects that have been confirmed or validated as planets—a step up from the planet candidate designation.

* koi_disposition: The disposition in the literature towards this exoplanet candidate. One of CANDIDATE, FALSE POSITIVE, NOT DISPOSITIONED or CONFIRMED.

* koi_pdisposition: The disposition Kepler data analysis has towards this exoplanet candidate. One of FALSE POSITIVE, NOT DISPOSITIONED, and CANDIDATE.

* koi_score: A value between 0 and 1 that indicates the confidence in the KOI disposition. For CANDIDATEs, a higher value indicates more confidence in its disposition, while for FALSE POSITIVEs, a higher value indicates less confidence in that disposition.


In [1]:
# # Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade
# # install joblib
# !pip install joblib

### Import Dependencies 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)
import tensorflow

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# Read/Load CSV file
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Basic Statistic Details

In [5]:
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,...,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,0.157059,0.244743,0.202975,0.125018,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,...,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,0.363882,0.429966,0.402243,0.330763,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,...,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,...,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,...,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,0.000000,0.000000,0.000000,0.000000,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,...,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,...,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,...,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


# Select Features (columns)
* Feature Selection: Removing irrelevant feature results in better performing model that is easeir to understands & model runs faster


In [6]:
target_names = df["koi_disposition"].unique()
#target_names
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


In [7]:
# Assign X (Independant data) and y (Dependant target)

# Set X equal to the entire data set, except for the first column
X = df.iloc[:, 1:]
# X.head()

# Set y equal to the first column
y = df.iloc[:,0].values.reshape(-1, 1)
# y.head()

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

# Search for top 10 features according to feature importances
model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

# sorted(zip(model.feature_importances_, X), reverse=True)

array([0.13073522, 0.16928087, 0.13071984, 0.05817844, 0.01773339,
       0.01305674, 0.02116328, 0.01249918, 0.02393155, 0.02871403,
       0.01103683, 0.0140373 , 0.00907399, 0.01992297, 0.02198486,
       0.02374409, 0.02089962, 0.01018814, 0.01162114, 0.0112719 ,
       0.00999627, 0.01020811, 0.01191154, 0.00855717, 0.00839419,
       0.00718397, 0.032982  , 0.009957  , 0.00896558, 0.02233331,
       0.01167389, 0.00973852, 0.00971656, 0.01824186, 0.00886778,
       0.01016445, 0.00851821, 0.01119689, 0.0108167 , 0.01078259])

In [9]:
# Store the top (10) features  as a series, using the column headers as the index
top_feat = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
top_feat

koi_fpflag_ss        0.169281
koi_fpflag_nt        0.130735
koi_fpflag_co        0.130720
koi_fpflag_ec        0.058178
koi_model_snr        0.032982
koi_time0bk_err2     0.028714
koi_time0bk_err1     0.023932
koi_duration_err2    0.023744
koi_steff_err1       0.022333
koi_duration_err1    0.021985
dtype: float64

In [10]:
# Set features based on feature importances
X = df[top_feat.index]

# Use `koi_disposition` for the y values
y = df['koi_disposition']

#y = df['koi_disposition'].values.reshape(-1, 1)

# Create  -- Train -- Test -- Split

In [11]:
from sklearn.model_selection import train_test_split

# Split the data into smaller buckets for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,koi_fpflag_ss,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ec,koi_model_snr,koi_time0bk_err2,koi_time0bk_err1,koi_duration_err2,koi_steff_err1,koi_duration_err1
6122,0,0,0,0,10.8,-0.008440,0.008440,-0.3060,154,0.3060
6370,1,0,0,1,13.8,-0.007950,0.007950,-0.2820,158,0.2820
2879,0,1,0,0,254.3,-0.006190,0.006190,0.0000,151,0.0000
107,0,0,0,0,38.4,-0.001820,0.001820,-0.0595,76,0.0595
29,0,0,0,0,696.5,-0.000083,0.000083,-0.0075,77,0.0075


In [12]:
# X and y Train shape have 5243 rows (80% of data)
X_train.shape, y_train.shape

((5243, 10), (5243,))

In [13]:
# X and y Test shape have 1748 rows (20% of data)
X_test.shape, y_test.shape

((1748, 10), (1748,))

# Pre-processing

Scale the data using the MinMaxScaler: 

MinMaxScaler: 
              * A way to normalize the input features/variables
              * Features will be transformed into the range
              * Scales the range of fetures from 0 to 1


In [14]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

#Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#print(np.matrix(X_test_scaled))

# Train the Model 
* Used TensorFlow- Deep Learning



In [15]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# ==================== Step 1: Label-encode data set =============================
label_encoder = LabelEncoder()

label_encoder.fit(y_train)

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# ==================== Step 2: Convert encoded labels to ONE-HOT-encoding ====================
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)


print(X_train_scaled.shape)

(5243, 10)


# Deep Learning Model

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a modle and  Neural Network Layers here
model = Sequential()

# 1st Layer (Input)
model.add(Dense(units=100, activation='relu', input_dim=10))

# 2nd Hidden Layer
model.add(Dense(units=80, activation='relu'))
          
# 3rd Layer (Output)          
model.add(Dense(units=3, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_11 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 243       
Total params: 9,423
Trainable params: 9,423
Non-trainable params: 0
_________________________________________________________________


In [28]:
# ==================== Compile the model ====================
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ==================== Fit the model ====================
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
164/164 - 1s - loss: 0.5479 - accuracy: 0.7181
Epoch 2/100
164/164 - 0s - loss: 0.3724 - accuracy: 0.8037
Epoch 3/100
164/164 - 0s - loss: 0.3512 - accuracy: 0.8322
Epoch 4/100
164/164 - 0s - loss: 0.3389 - accuracy: 0.8428
Epoch 5/100
164/164 - 0s - loss: 0.3305 - accuracy: 0.8436
Epoch 6/100
164/164 - 0s - loss: 0.3264 - accuracy: 0.8468
Epoch 7/100
164/164 - 0s - loss: 0.3205 - accuracy: 0.8516
Epoch 8/100
164/164 - 0s - loss: 0.3178 - accuracy: 0.8556
Epoch 9/100
164/164 - 0s - loss: 0.3177 - accuracy: 0.8568
Epoch 10/100
164/164 - 0s - loss: 0.3169 - accuracy: 0.8558
Epoch 11/100
164/164 - 0s - loss: 0.3156 - accuracy: 0.8543
Epoch 12/100
164/164 - 0s - loss: 0.3148 - accuracy: 0.8526
Epoch 13/100
164/164 - 0s - loss: 0.3123 - accuracy: 0.8591
Epoch 14/100
164/164 - 0s - loss: 0.3175 - accuracy: 0.8554
Epoch 15/100
164/164 - 0s - loss: 0.3131 - accuracy: 0.8554
Epoch 16/100
164/164 - 0s - loss: 0.3132 - accuracy: 0.8592
Epoch 17/100
164/164 - 0s - loss: 0.3145 - accura

## Quantify our Trained Model "Evaluate"

In [29]:
# ==================== Evaluate ====================
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")


55/55 - 0s - loss: 0.3174 - accuracy: 0.8581
Normal Neural Network - Loss: 0.3174290657043457, Accuracy: 0.8581235408782959


## Prediction

In [30]:
# ==================== Prediction ====================

# Use the first 5 test data values to make a prediction and compare it to the actual labels
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

# print(f"Predicted classes : {prediction_labels}")
# print(f"Actual Labels: {list(y_test[:10])}")

df_predict = pd.DataFrame({"Actual":y_test[:10], "Predicted":prediction_labels}) 
df_predict.head()

,Actual,Predicted
4982,FALSE POSITIVE,FALSE POSITIVE
4866,CANDIDATE,CANDIDATE
2934,FALSE POSITIVE,FALSE POSITIVE
5007,FALSE POSITIVE,FALSE POSITIVE
3869,FALSE POSITIVE,FALSE POSITIVE


In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test[:10], prediction_labels)

array([[3, 1, 0],
       [0, 1, 0],
       [0, 0, 5]], dtype=int64)

## SAVE the Model

In [20]:
model.save("DeepLearning_model.h5")

## ?????????????

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [74]:
from tensorflow.keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

# Check Sequential parameters that can be used for Tuning
model = Sequential()

# 1st Layer (Input)
model.add(Dense(units=150, activation='relu', input_dim=10))

# 2nd Hidden Layer
model.add(Dense(units=90, activation='sigmoid'))
          
# 3rd Layer (Output)          
model.add(Dense(units=3, activation='softmax'))

model.summary()

# ==================== Compile the model ====================
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 150)               1650      
_________________________________________________________________
dense_23 (Dense)             (None, 90)                13590     
_________________________________________________________________
dense_24 (Dense)             (None, 3)                 273       
Total params: 15,513
Trainable params: 15,513
Non-trainable params: 0
_________________________________________________________________


In [106]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score

# scoring = ['precision_macro', 'recall_macro', 'balanced_accuracy']
# param_grid = dict(epochs=[10,20,30])
# grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, n_jobs=-1, cv=5, refit=False, return_train_score=False)

# batchSize = [10,30,50,70,90,130]
# epochs = [10,20,30]

# scores = {
#         'precision_score': make_scorer(precision_score),
#         'recall_score': make_scorer(recall_score),
#         'accuracy_score': make_scorer(accuracy_score)
#         }

# param_grid = dict(epochs=epochs, batch_size=batchSize)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3 , scoring='accuracy', refit='boolean')
def getModel(optimizer):
    model = Sequential()
 
    model.add(Dense(units=150, activation='relu', input_dim=10))

    model.add(Dense(units=90, activation='sigmoid'))
       
    model.add(Dense(units=3, activation='softmax'))
    
    model.compile(optimizer=optimizer , loss = tensorflow.losses.categorical_crossentropy , metrics=['accuracy'])
    return model

optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
epochs = [10, 50, 100]

param_grid = dict(epochs=epochs, optimizer=optimizer)

Kmodel = KerasClassifier(build_fn=getModel, verbose=1)
grid = GridSearchCV(estimator=Kmodel, param_grid=param_grid, scoring='accuracy', n_jobs=-1, refit='boolean')

In [107]:
# Train the model with GridSearch

grid.fit(X_train_scaled, y_train)

Epoch 1/100
164/164 [==============================] - 0s 823us/step - loss: 0.7906 - accuracy: 0.6097
Epoch 2/100
164/164 [==============================] - 0s 795us/step - loss: 0.3857 - accuracy: 0.7743
Epoch 3/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3788 - accuracy: 0.8079
Epoch 4/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3483 - accuracy: 0.8324
Epoch 5/100
164/164 [==============================] - 0s 881us/step - loss: 0.3411 - accuracy: 0.8424
Epoch 6/100
164/164 [==============================] - 0s 771us/step - loss: 0.3399 - accuracy: 0.8407
Epoch 7/100
164/164 [==============================] - 0s 789us/step - loss: 0.3310 - accuracy: 0.8416
Epoch 8/100
164/164 [==============================] - 0s 759us/step - loss: 0.3318 - accuracy: 0.8480
Epoch 9/100
164/164 [==============================] - 0s 893us/step - loss: 0.3136 - accuracy: 0.8570
Epoch 10/100
164/164 [==============================] - 0s 1ms/step - loss: 0

164/164 [==============================] - 0s 881us/step - loss: 0.3042 - accuracy: 0.8610
Epoch 81/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3186 - accuracy: 0.8527
Epoch 82/100
164/164 [==============================] - 0s 985us/step - loss: 0.3004 - accuracy: 0.86160s - loss: 0.3002 - accuracy: 0.86
Epoch 83/100
164/164 [==============================] - 0s 826us/step - loss: 0.2911 - accuracy: 0.8707
Epoch 84/100
164/164 [==============================] - 0s 936us/step - loss: 0.3001 - accuracy: 0.8645
Epoch 85/100
164/164 [==============================] - 0s 857us/step - loss: 0.3083 - accuracy: 0.8601
Epoch 86/100
164/164 [==============================] - 0s 893us/step - loss: 0.2968 - accuracy: 0.8570
Epoch 87/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3031 - accuracy: 0.8571
Epoch 88/100
164/164 [==============================] - 0s 2ms/step - loss: 0.3021 - accuracy: 0.8614
Epoch 89/100
164/164 [===========================

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001DE88C18B88>,
             iid='warn', n_jobs=-1,
             param_grid={'epochs': [10, 50, 100],
                         'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta',
                                       'Adam', 'Adamax', 'Nadam']},
             pre_dispatch='2*n_jobs', refit='boolean', return_train_score=False,
             scoring='accuracy', verbose=0)

In [108]:
# List the best parameters for this dataset
print(grid.best_params_)

# List the best score
print('Best Score: %.3f' % grid.best_score_)

{'epochs': 100, 'optimizer': 'Adam'}
Best Score: 0.859


In [109]:
# Score the model
print('Model Score: %.3f' % grid.score(X_test_scaled, y_test))

55/55 [==============================] - 0s 610us/step
Model Score: 0.855


In [110]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_grid.head()

55/55 [==============================] - 0s 794us/step


,Actual,Predicted
4982,FALSE POSITIVE,FALSE POSITIVE
4866,CANDIDATE,CANDIDATE
2934,FALSE POSITIVE,FALSE POSITIVE
5007,FALSE POSITIVE,FALSE POSITIVE
3869,FALSE POSITIVE,FALSE POSITIVE


In [111]:
# Calculate classification report
# print(np.array(y_test))

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=target_names))

                precision    recall  f1-score   support

     CONFIRMED       0.72      0.67      0.70       411
FALSE POSITIVE       0.74      0.76      0.75       484
     CANDIDATE       0.98      1.00      0.99       853

      accuracy                           0.86      1748
     macro avg       0.81      0.81      0.81      1748
  weighted avg       0.85      0.86      0.85      1748



# Save the Model

In [112]:
import joblib
filename = 'DeepLearning_model.sav'
joblib.dump(Sequential, filename)

['DeepLearning_model.sav']